In [2]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms

# Basic autograd example 1

In [3]:
# Create tensors
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# Build a computational graph
y = w * x + b

# Compute gradients
y.backward()

# Print the gradients
print(x.grad)
print(w.grad)
print(b.grad)

tensor(2.)
tensor(1.)
tensor(1.)


# Basic autograd example 2

In [4]:
# Create tensor of shape (10, 3) and (10, 2)
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build fully connected layer
linear = nn.Linear(3, 2)
print('w', linear.weight)
print('b', linear.bias)

w Parameter containing:
tensor([[-0.2043, -0.0936, -0.4224],
        [ 0.2401, -0.1333,  0.3975]], requires_grad=True)
b Parameter containing:
tensor([-0.0187,  0.0294], requires_grad=True)


In [5]:
# Build loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass
pred = linear(x)

# Compute loss
loss = criterion(pred, y)
print(loss)

tensor(0.8993, grad_fn=<MseLossBackward>)


In [6]:
# Backward pass
loss.backward()

print('dL/dw', linear.weight.grad)
print('dL/db', linear.bias.grad)

dL/dw tensor([[-0.1896, -0.1143, -0.0868],
        [-0.1112, -0.7044,  0.4858]])
dL/db tensor([-0.4438,  0.0749])


In [7]:
# 1-step gradient descent
# This is equivalent to:
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)
optimizer.step()

In [8]:
# Print loss after 1-step gradient descent
pred = linear(x)
loss = criterion(pred, y)
print('Loss after 1-step optimization:', loss.item())

Loss after 1-step optimization: 0.8892849683761597


# Loading data from numpy

In [9]:
# Create numpy array
x = np.array([[1, 2], [3, 4]])

# Convert the numpy array to a torch tensor
y = torch.from_numpy(x)

# Covert the torch tensor to a numpy array
z = y.numpy()

# Input pipline

In [10]:
# Download and construct CIFAR-10 dataset
train_dataset = torchvision.datasets.CIFAR10(root='./data', 
                                             train=True, 
                                             transform=transforms.ToTensor(), 
                                             download=True)

# Fetch one data pair (read from disk)
image, label = train_dataset[0]
print(image.size())
print(label)

0it [00:00, ?it/s]

 99%|█████████▉| 169058304/170498071 [00:16<00:00, 12069812.06it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
torch.Size([3, 32, 32])
6


170500096it [00:29, 12069812.06it/s]                               

In [11]:
# Data loader (this provides queues and threads in a very simple way)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files
data_iter = iter(train_loader)

# Mini-batch images and labels
image, label = data_iter.next()

# Actual usage of the data loader is as below
for images, labels in train_loader:
    pass

# Input pipline for custom dataset

In [12]:
# You should build your custom dataset as below
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # TODO
        # 1. Initialize file paths or a list of file names
        pass
    def __getitem__(self, index):
        # TODO
        # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open)
        # 2. Preprocess the data (e.g. torchvision.Transform)
        # 3. Return a data pair (e.g. image and label)
        pass
    def __len__(self):
        # You should change 0 to the total size of your dataset
        return 0
    
# You can then use the prebuilt data loader
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)


ValueError: num_samples should be a positive integer value, but got num_samples=0

# Pretrained model

In [14]:
# Download and load the pretrained ResNet-18
resnet = torchvision.models.resnet18(pretrained=True)

# If you want to finetune only the top layer of the model, set as below
for param in resnet.parameters():
    param.requires_grad = False  # Freeze the params
    
# Replace the top layer for finetuning
resnet.fc = nn.Linear(resnet.fc.in_features, 100)  # 100 is an exmaple

# Forward pass
images = torch.randn(64, 3, 224, 224)  # Generate 64 224*224 images with 3 channels
outputs = resnet(images)
print(outputs.size())

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /hpc/home/ephyan/.cache/torch/checkpoints/resnet18-5c106cde.pth

  0%|          | 0.00/44.7M [00:00<?, ?B/s]
  3%|▎         | 1.46M/44.7M [00:00<00:02, 15.3MB/s]
  9%|▊         | 3.88M/44.7M [00:00<00:02, 17.3MB/s]
 11%|█         | 4.92M/44.7M [00:00<00:03, 13.3MB/s]
 17%|█▋        | 7.63M/44.7M [00:00<00:02, 15.8MB/s]
 21%|██        | 9.44M/44.7M [00:00<00:02, 16.6MB/s]
 25%|██▍       | 11.1M/44.7M [00:00<00:02, 16.7MB/s]
 28%|██▊       | 12.7M/44.7M [00:00<00:01, 16.9MB/s]
 35%|███▌      | 15.6M/44.7M [00:00<00:01, 19.5MB/s]
 40%|███▉      | 17.8M/44.7M [00:00<00:01, 20.2MB/s]
 47%|████▋     | 20.8M/44.7M [00:01<00:01, 22.6MB/s]
 52%|█████▏    | 23.1M/44.7M [00:01<00:01, 20.8MB/s]
 59%|█████▉    | 26.2M/44.7M [00:01<00:00, 23.2MB/s]
 64%|██████▍   | 28.7M/44.7M [00:01<00:00, 18.3MB/s]
 69%|██████▉   | 30.7M/44.7M [00:01<00:00, 16.8MB/s]
 73%|███████▎  | 32.5M/44.7M [00:01<00:00, 15.5MB/s]
 77%|███████▋  | 34.

torch.Size([64, 100])


# Save and load model

In [15]:
# Save and load the entire model
torch.save(resnet, 'model.ckpt')
model = torch.load('model.ckpt')

# Save and load only the model parameters (recommended)
torch.save(resnet.state_dict(), 'params.ckpt')
resnet.load_state_dict(torch.load('params.ckpt'))

<All keys matched successfully>